In [ ]:
import itertools
from functools import cache
from pathlib import Path

type Position = tuple[int, int]
type Pad = dict[Position, str]

In [ ]:
DOOR_CODES = Path("day21_input.txt").read_text().strip().split("\n")

NUM_PAD = dict(zip(itertools.product(range(4), range(3)), "789456123 0A", strict=True))
DIR_PAD = dict(zip(itertools.product(range(2), range(3)), " ^A<v>", strict=True))

In [ ]:
def find_paths(start: str, end: str, pad: Pad) -> list[str]:
    """Find the shortest path between start and end on a pad.

    We group all the horisontal/vertical moves together, to minimize the number of
    turns. The returned list will have either 1 or 2 possible paths.
    """
    start_row, start_col = next(k for k, v in pad.items() if v == start)
    end_row, end_col = next(k for k, v in pad.items() if v == end)
    blank_pos = next(k for k, v in pad.items() if v == " ")

    # Number of steps to move vertically and horizontally
    vert = ("v" if end_row > start_row else "^") * abs(end_row - start_row)
    hori = (">" if end_col > start_col else "<") * abs(end_col - start_col)

    # If we only need to move in one direction, return that path
    if not vert:
        return [f"A{hori}A"]
    if not hori:
        return [f"A{vert}A"]

    # Add both "diagonal" options, unless they pass over the blank
    paths = []
    if (end_row, start_col) != blank_pos:
        paths.append(f"A{vert}{hori}A")
    if (start_row, end_col) != blank_pos:
        paths.append(f"A{hori}{vert}A")

    return paths

In [ ]:
@cache
def robot_moves(sequence, num_layers=2) -> int:
    """Find the minimum number of robot moves needed to enter a sequence."""
    if num_layers == 0:
        # Subtract 1, since the sequence has an extra A at the beginning
        return len(sequence) - 1
    num_moves = 0
    for start, end in itertools.pairwise(sequence):
        num_moves += min(
            robot_moves(subseq, num_layers - 1)
            for subseq in find_paths(start, end, DIR_PAD)
        )
    return num_moves

In [ ]:
def num_moves_for_code(door_code: str, num_layers: int = 2) -> int:
    """Find the minimum number of robot moves needed to enter a door code.

    Since every robot move starts and ends with an "A", we can compute every move
    individually and add them up. Using short sequences, we can cache the results.
    """
    moves = 0
    for start, end in itertools.pairwise("A" + door_code):
        moves += min(
            robot_moves(subseq, num_layers)
            for subseq in find_paths(start, end, NUM_PAD)
        )
    return moves

# Part 1


In [ ]:
answer = 0
for door_code in DOOR_CODES:
    answer += num_moves_for_code(door_code) * int(door_code.rstrip("A"))
answer

In [ ]:
robot_moves.cache_info()

# Part 2


In [ ]:
answer = 0
for door_code in DOOR_CODES:
    answer += num_moves_for_code(door_code, 25) * int(door_code.rstrip("A"))
answer

In [ ]:
robot_moves.cache_info()